Purpose of this notebook is to check for the significance of parameters on the SEQ and WAL gas prices
The current parameters for consideration are 
- Demand/Supply (VIC,NSW,QLD,LNG [QLD]) - skipped for now
- Temperature in VIC 
- Temp in NSW 
- Prices (VIC/NSW)
- Prices (JKM - spot cargos, JCC - spas)

In [122]:
import pandas as pd 
import numpy as np 
import sklearn as sk 
import math 
import seaborn as sns 

In [123]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

Loading the relevant data 

In [124]:
## Allows us to calculate the VWAP at SEQ and WAL 

trades = pd.read_excel('Gas_Data/GSH Historical Trades.xlsx')
trades = trades.replace(to_replace = 'WAL Non-netted', value = 'WAL')

In [142]:
## Creates VWAP for SEQ and WAL in trades df

trades ['v*p'] = trades['DAILY_QTY_GJ'] * trades['TRADE_PRICE']
tester = trades.groupby(['TRADE_DATE', 'LOCATION'], as_index= False).sum()
tester['VWAP'] = tester['v*p']/tester['DAILY_QTY_GJ']
tester.drop( labels = ['TRADE_PRICE', 'DAILY_QTY_GJ', 'v*p'] , axis = 1, inplace=True)


WAL = tester[tester ['LOCATION'] == 'WAL'].copy()
WAL.rename(columns={ "LOCATION": "WAL", 'VWAP': "VWAP(WAL)"}, inplace= True )

SEQ = tester[tester ['LOCATION'] == 'SEQ'].copy()
SEQ.rename(columns={ "LOCATION": "SEQ", 'VWAP': "VWAP(SEQ)"}, inplace= True )

trades_df = WAL.merge(SEQ, on = 'TRADE_DATE', suffixes = (None, None), how = 'outer')


In [126]:
## Price data for SYD,ADL and BRISVEGAS markets
## Expost price is used

sttm_price = pd.read_excel("Gas_Data/STTM Price and Withdrawals.xlsx", sheet_name = 'Deviation Prices')
sttm_price = sttm_price[['DateTime','expost_imbalance_price','hub_name']]
sttm_price.tail()

,DateTime,expost_imbalance_price,hub_name
8851,2022-11-29,24.2321,BRI
8852,2022-11-29,21.8000,SYD
8853,2022-11-30,24.8000,ADL
8854,2022-11-30,24.0111,BRI
8855,2022-11-30,23.5711,SYD


In [127]:
## VIC Market demand and price data 
## Note that the expost price is used with the 6am price being the reference price 
## Skip demand/supply for now. 

dwgm_price = pd.read_excel("Gas_Data/DWGM Prices and Demand.xlsx", sheet_name='Prices')
dwgm_price.head()

# dwgm_demand = pd.read_excel("DWGM Prices and Demand.xlsx", sheet_name='Demand')
# dwgm_demand[dwgm_demand['Gas_Date'] == '2022-12-13']
# dwgm_demand.tail()

,Gas_Date,Hour,Price
0,2007-02-01,6,3.355
1,2007-02-01,10,3.380
2,2007-02-01,14,3.380
3,2007-02-01,18,3.355
4,2007-02-01,22,3.355


In [128]:
## Function to convert date into correct format for Temp dataframes 
def create_date(year, month, day):
    output = str(year) +"/"+ str(month) +"/"+ str(day)

    return output 

In [129]:
syd_mtemp = pd.read_csv('Weather_Data/syd(observatory_hill)_maxtemp.csv')
mel_mtemp = pd.read_csv('Weather_Data/melb(olympic_park)_maxtemp.csv')

syd_mtemp['Date'] = syd_mtemp.apply(lambda row: create_date(row['Year'], row['Month'], row['Day']), axis = 1)
syd_mtemp['Date']= pd.to_datetime(syd_mtemp['Date'])
syd_mtemp = syd_mtemp[['Date', 'Maximum temperature (Degree C)']]
syd_mtemp.rename(columns={'Maximum temperature (Degree C)': 'Syd_Temp'}, inplace= True)


mel_mtemp['Date'] = mel_mtemp.apply(lambda row: create_date(row['Year'], row['Month'], row['Day']), axis = 1)
mel_mtemp['Date']= pd.to_datetime(mel_mtemp['Date'])
mel_mtemp = mel_mtemp[['Date', 'Maximum temperature (Degree C)']]
mel_mtemp.rename(columns={'Maximum temperature (Degree C)': 'Mel_Temp'}, inplace= True)

# syd_mtemp.tail()
# mel_mtemp.tail()

In [130]:
mel_mtemp.head()

,Date,Mel_Temp
0,2013-01-01,NaN
1,2013-01-02,NaN
2,2013-01-03,NaN
3,2013-01-04,NaN
4,2013-01-05,NaN


In [131]:
syd_mtemp.head()

,Date,Syd_Temp
0,2017-01-01,NaN
1,2017-01-02,NaN
2,2017-01-03,NaN
3,2017-01-04,NaN
4,2017-01-05,NaN


Adding in the JCC, JKM (ICE and Platt), and Dated Brent

In [132]:
markers = pd.read_excel('JKM_JCC(Oliver).xlsx',  sheet_name= 'Source Data_Historicals')
markers = markers.iloc[:,[0,2,3,9,10,12,13]] ## Recovers columns corresponding to ICE JKM, Platts JKM, JCC, and Dated Brent (and the respective datetime columns too)

markers.head()

,Unnamed: 0,ICE JKM,Platts JKM,Unnamed: 9,JCC,Unnamed: 12,Dated Brent
0,Series Function,ICEOTC.JKM,AAOVQ00,Series Function,PAJ.CRUDE.OIL.USD,Series Function,PCAAS00
1,NaN,Close,Index (USD/MMBTU),NaN,Close,NaN,Close (USD/BBL)
2,NaN,"ICE OTC, JKM, LNG Futures",Platts: LNG Japan/Korea DES Spot Crg (DW USD/MMB),NaN,"Petroleum Association of Japan, Oil Import Pri...",NaN,Platts: Dated Brent (DW USD/BBL)
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-05 00:00:00,32.158,36.243,2022-09-01 00:00:00,110.7882,2022-12-05 00:00:00,85.59


In [133]:
jkms = markers.iloc[:,[0,1,2]]
jcc =  markers.iloc[:,[3,4]]
dbrent =  markers.iloc[:,[5,6]]

dbrent = dbrent.drop(labels = [0,1,2,3], axis = 0)
jkms = jkms.drop(labels = [0,1,2,3], axis = 0)
jcc = jcc.drop(labels = [0,1,2,3], axis = 0)

In [134]:

dbrent.columns = ['Date', 'Dated_Brent']
dbrent['Date'] = pd.to_datetime(dbrent['Date'])
dbrent.head(10)

,Date,Dated_Brent
4,2022-12-05,85.59
5,2022-12-02,86.945
6,2022-12-01,89.08
7,2022-11-30,86.54
8,2022-11-29,83.94
9,2022-11-28,82.65
10,2022-11-25,85.475
11,2022-11-24,85.485
12,2022-11-23,85.315
13,2022-11-22,89.4


In [135]:

jkms.columns = ['Date', 'ICE_JKM', 'Platt_JKM']
jkms['Date'] = pd.to_datetime(jkms['Date'])
jkms.head()

,Date,ICE_JKM,Platt_JKM
4,2022-12-05,32.158,36.243
5,2022-12-02,32.286,32.765
6,2022-12-01,31.417,38.075
7,2022-11-30,33.75,33.505
8,2022-11-29,31.331,31.149


In [136]:

jcc.columns = ['Date', 'JCC']
jcc['Date'] = pd.to_datetime(jcc['Date'])
jcc.head()

,Date,JCC
4,2022-09-01,110.7882
5,2022-08-01,112.4639
6,2022-07-01,116.3807
7,2022-06-01,116.9194
8,2022-05-01,107.8182


Merging all the data to the same dataframe

In [143]:
trades_df.head()

,TRADE_DATE,WAL,VWAP(WAL),SEQ,VWAP(SEQ)
0,2016-07-06,WAL,9.169492,NaN,NaN
1,2016-07-07,WAL,10.000000,NaN,NaN
2,2016-07-25,WAL,10.000000,NaN,NaN
3,2016-08-05,WAL,6.000000,NaN,NaN
4,2016-08-29,WAL,6.500000,NaN,NaN


In [148]:
df = jkms.merge(syd_mtemp, left_on = 'Date', right_on = 'Date')
df = df.merge(mel_mtemp, left_on = 'Date', right_on = 'Date')
df = df.merge(sttm_price[sttm_price['hub_name'] == 'SYD'], left_on= 'Date', right_on= 'DateTime')
df = df.merge(dwgm_price[dwgm_price['Hour'] == 6], left_on= 'Date', right_on= 'Gas_Date')
df = df.merge(trades_df[['TRADE_DATE', 'VWAP(WAL)', 'VWAP(SEQ)']], left_on='Date', right_on= 'TRADE_DATE')


df = df.drop(columns=['DateTime', 'TRADE_DATE', 'hub_name', 'Gas_Date', 'Hour'])
df = df.rename(columns ={'expost_imbalance_price': 'Syd_Price', 'Price': 'Mel_Price'})

df
# pd.concat([jkms, syd_mtemp],join = 'inner')

,Date,ICE_JKM,Platt_JKM,Syd_Temp,Mel_Temp,Syd_Price,Mel_Price,VWAP(WAL),VWAP(SEQ)
0,2022-11-30,33.75,33.505,22.9,18.4,23.5711,22.9900,23.241667,NaN
1,2022-11-29,31.331,31.149,25.4,18.4,21.8000,21.8011,22.065789,NaN
2,2022-11-28,30.265,31.029,23.8,18.8,22.0000,20.5011,21.678571,22.0000
3,2022-11-25,29.954,31.577,24.1,20.6,22.0000,22.0156,19.646552,18.9500
4,2022-11-24,30.078,31.368,25.4,19.2,22.0000,22.2851,20.570319,18.8375
...,...,...,...,...,...,...,...,...,...
1415,2017-03-23,5.382,5.35,NaN,25.1,9.7382,7.5900,8.500000,NaN
1416,2017-03-10,6.086,5.9,NaN,23.8,11.3900,10.4690,9.100000,NaN
1417,2017-03-09,6.096,5.95,NaN,32.3,11.3900,10.5000,9.100000,NaN
1418,2017-03-08,6.121,6.05,NaN,31.5,12.3000,10.5500,8.200000,NaN


Correlation Heatmap

In [ ]:
train_data = 
train_data['Target'] = target

C_mat = train_data.corr()
fig = plt.figure(figsize = (15,15))

sb.heatmap(C_mat, vmax = .8, square = True)
plt.show()